In [1]:
import pandas as pd
import docx
import re
import numpy as np
from math import floor

In [2]:
def parse_document(doc_path, skill_path, tier_start=-1):
    doc = docx.Document(doc_path)
    paras = [x.text for x in doc.paragraphs]
    for i in range(len(paras)):
        if paras[i] == ' ':
            paras[i] = ''
    particular_value = ''
    result = []
    temp_list = []
    for i in paras:
        if i == particular_value:
            temp_list.append(i)
            result.append(temp_list)
            temp_list = []
        else:
            temp_list.append(i)
    result.append(temp_list)
    df = pd.DataFrame({'text':result})
    tier_list = []
    ability_list = []
    tier = tier_start
    for _, row in df.iterrows():
        data = row['text']
        if len(data) > 1:
            if data[0].split(' ')[0] == 'Tier':
                tier += 1
                data.pop(0)
            tier_list.append(tier)
            ability_list.append(data)
    ability_df = pd.DataFrame({'Ability':ability_list, 'Tier':tier_list})
    ability_df['Path'] = skill_path
    ability_df['Skill Name'] = ability_df.Ability.apply(lambda x:x[0].split(':')[0])
    try:
        ability_df['Description'] = ability_df.Ability.apply(lambda x:x[0].split(':')[1])
    except:
        print(ability_df.Ability)
    pr = []
    lim = []
    preq = []
    aug = []
    for _ , row in ability_df.iterrows():
        for sublist in row['Ability']:
            if 'Phys Rep' in sublist:
                pr.append(sublist.split(':')[1])
            if 'Limitations' in sublist:
                lim.append(sublist.split(':')[1])
            if 'Prerequisite' in sublist:
                preq.append(sublist.split(':')[1])
            if 'Augment' in sublist:
                aug.append(sublist.split(':')[1])
        if 'Phys Rep' not in str(row.Ability):
            pr.append(None)
        if 'Limitations' not in str(row.Ability):
            lim.append(None)
        if 'Prerequisite' not in str(row.Ability):
            preq.append(None)
        if 'Augment' not in str(row.Ability):
            aug.append(None)
    ability_df['Phys Rep'] = pr
    ability_df['Limitations'] = lim
    ability_df['Prerequisite'] = preq
    ability_df['Augment'] = aug
    ability_df = ability_df.map(lambda x: x.strip() if isinstance(x, str) else x)
    return ability_df[['Skill Name', 'Description', 'Path', 'Tier', 'Limitations', 'Phys Rep', 'Prerequisite', 'Augment']]

In [3]:
artificer_df = parse_document("The Artificer's Path.docx", 'Artificer', tier_start=0)

In [4]:
artificer_df

,Skill Name,Description,Path,Tier,Limitations,Phys Rep,Prerequisite,Augment
0,Appraise,"Upon taking the Appraise skill, an artificer c...",Artificer,1,"In terms of roleplay, your keen eye for object...","To Appraise - a magnifying glass, examiner’s l...",None,None
1,Appraise [Gems],"Upon taking the Appraise skill, an artificer c...",Artificer,1,"In terms of roleplay, your keen eye for object...","To Appraise - a magnifying glass, examiner’s l...",None,None
2,Appraise [Runes],"Upon taking the Appraise skill, an artificer c...",Artificer,1,"In terms of roleplay, your keen eye for object...","To Appraise - a magnifying glass, examiner’s l...",None,None
3,Appraise [Weaponry],"Upon taking the Appraise skill, an artificer c...",Artificer,1,"In terms of roleplay, your keen eye for object...","To Appraise - a magnifying glass, examiner’s l...",None,None
4,Appraise [Armor],"Upon taking the Appraise skill, an artificer c...",Artificer,1,"In terms of roleplay, your keen eye for object...","To Appraise - a magnifying glass, examiner’s l...",None,None
5,Appraise [Spell Focus],"Upon taking the Appraise skill, an artificer c...",Artificer,1,"In terms of roleplay, your keen eye for object...","To Appraise - a magnifying glass, examiner’s l...",None,None
6,Appraise [Alchemy],"Upon taking the Appraise skill, an artificer c...",Artificer,1,"In terms of roleplay, your keen eye for object...","To Appraise - a magnifying glass, examiner’s l...",None,None
7,Appraise [Machinery],"Upon taking the Appraise skill, an artificer c...",Artificer,1,"In terms of roleplay, your keen eye for object...","To Appraise - a magnifying glass, examiner’s l...",None,None
8,Appraise [Clothing],"Upon taking the Appraise skill, an artificer c...",Artificer,1,"In terms of roleplay, your keen eye for object...","To Appraise - a magnifying glass, examiner’s l...",None,None
9,Appraise [Art],"Upon taking the Appraise skill, an artificer c...",Artificer,1,"In terms of roleplay, your keen eye for object...","To Appraise - a magnifying glass, examiner’s l...",None,None


In [5]:
warrior_df = parse_document("The Warrior's Path.docx", 'Warrior')
rogue_df = parse_document("The Rogue's Path.docx", 'Rogue')
healer_df = parse_document("The Healer's Path.docx", 'Healer')
mage_df = parse_document("The Mage's Path.docx", 'Mage')
bard_df = parse_document("The Bard's Path.docx", 'Bard', tier_start=0)
artificer_df = parse_document("The Artificer's Path.docx", 'Artificer', tier_start=0)

In [6]:
skills = pd.concat([warrior_df,rogue_df,healer_df,mage_df,bard_df, artificer_df])

In [7]:
skills['Spell'] = skills['Description'].apply(lambda x: x.startswith(('(Spell)', '(Combat Magic)')))

In [8]:
skills

,Skill Name,Description,Path,Tier,Limitations,Phys Rep,Prerequisite,Augment,Spell
0,Basic Weapon Proficiency,Players learn the basic combat and safety rule...,Warrior,0,None,None,None,None,False
1,Armor Proficiency,Players learn about the armor that they wear a...,Warrior,0,None,None,None,None,False
2,Kindle Flame/Torch,Player gains proficiency at creating normal fi...,Warrior,0,10’ radius,Optional (larp safe electronically lighted tor...,None,None,False
3,Shield Control,Players learn the art of Shield Control and ma...,Warrior,0,None,None,None,None,False
4,Advanced Weapon Training,Study and become proficient in the use of two-...,Warrior,1,None,None,None,None,False
...,...,...,...,...,...,...,...,...,...
9,Appraise [Art],"Upon taking the Appraise skill, an artificer c...",Artificer,1,"In terms of roleplay, your keen eye for object...","To Appraise - a magnifying glass, examiner’s l...",None,None,False
10,Tinkerer’s Quirk,You may attach a mechanism to an object that a...,Artificer,1,None,"an actual light emitting material, a picture, ...",None,None,False
11,Schematic Encryption,You may disguise the meaning of your Artificer...,Artificer,1,None,A Schematic (see The Object’s Schematics below...,None,None,False
12,Cooperative Action [Artificer],The best Artificers learn that they can get fu...,Artificer,1,Keep in mind that if you encrypt a Schematic a...,All Artificers must be within touch distance o...,Schematic Encryption,None,False


In [9]:
skills.to_excel('Skills_Table.xlsx', index=False)

In [10]:
skills.to_csv('Skills_Table.csv', index=False)

In [11]:
# skills['Uses'] = skills['Limitations'].apply(lambda x: re.findall(r'[^.]* per [^.]*\.', x, re.IGNORECASE) if isinstance(x, str) else x)
# skills[~((skills['Uses'].isna()) | (skills['Uses'].astype(str) == '[]'))].to_excel('Skill Use.xlsx', index=False)

In [12]:
use_df = pd.read_excel('Skill Use.xlsx')

In [13]:
df = skills
known = ['Read/Write Arcana', 'First Aid', 'Basic Weapon Proficiency', 'Armor Proficiency', 'Darkness', 'Light', 'Kindle Flame/Torch', 'Healer’s Advanced Weapon Proficiency', 'Shield Control', 'Sanctify/Defile Location', 'Mage’s Staff', 'Holy Armor', 'Sense Blessed/Cursed', 'Divine Blessed/Cursed', 'Console', 'Remove Curse/Blessing', 'Scrolls & Potions', 'Forstall Death', 'Bless/Curse Weapon/Item', 'Smite', 'Create/Destroy Holy/Unholy Relic Weapon/Item', 'Heal Minor', 'Heal Major', 'Speak With Dead']
known_data = df[df['Skill Name'].isin(known)]

In [14]:
tier_df = pd.DataFrame({'Path':['Warrior', 'Rogue', 'Healer', 'Mage', 'Bard', 'Artificer'], 'Tier':[0,0,0,0,0,0]})
tier_df = pd.concat([known_data, tier_df]).groupby('Path')['Tier'].max().reset_index()

In [18]:
def use_calc(path, base, mod, unit):
    tier = tier_df[tier_df['Path'] == path].iloc[0]['Tier']
    use_count = base + eval(str(mod).replace('t', str(tier)))
    return f'{use_count} {unit}', use_count

In [19]:
use_df.apply(lambda x:use_calc(x['Path'], x['Base'], x['Tier Modifer'], x['Unit']), axis=1)

0       (3 per day, 3)
1       (3 per day, 3)
2       (3 per day, 3)
3       (0 per day, 0)
4       (0 per day, 0)
            ...       
91    (1 per event, 1)
92    (1 per event, 1)
93    (1 per event, 1)
94    (1 per event, 1)
95    (1 per event, 1)
Length: 96, dtype: object

In [21]:
use_df[['Uses', 'Use Count']] = pd.DataFrame(use_df.apply(lambda x:use_calc(x['Path'], x['Base'], x['Tier Modifer'], x['Unit']), axis=1).to_list())
use_df = use_df[['Skill Name', 'Path', 'Tier', 'Uses', 'Use Count']]
known_data = pd.merge(known_data, use_df, on=['Skill Name','Path','Tier'], how='left')
known_data.sort_values('Use Count', ascending=False).drop_duplicates('Skill Name').sort_index().sort_values('Tier')